Question 1. What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

In [68]:
import pandas as pd, requests, re, numpy as np
from io import StringIO

def get_withdrawn_ipos() -> pd.DataFrame:
    url = "https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/119.0 Safari/537.36"
        )
    }
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    return pd.read_html(StringIO(resp.text))[0]


In [69]:
withdrawn_raw = get_withdrawn_ipos()
withdrawn_raw.head()          # just to confirm: 100 rows × 4 cols


,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-


In [70]:
#Add Company Class

patterns = [
    (r"\bacquisition (corp|corporation)\b", "Acq.Corp"),
    (r"\binc(orporated)?\b",                "Inc"),
    (r"\bgroup\b",                          "Group"),
    (r"\b(ltd|limited)\b",                  "Limited"),
    (r"\bholdings?\b",                      "Holdings"),
]
def classify(name: str) -> str:
    low = str(name).lower()
    for pat, lbl in patterns:
        if re.search(pat, low):
            return lbl
    return "Other"

withdrawn_raw["Company Class"] = withdrawn_raw["Company Name"].map(classify)


#Compute Avg Price and clean Shares Offered

def avg_price(pr: str):
    if pd.isna(pr) or pr.strip() == "-":
        return np.nan
    vals = list(map(float, re.findall(r"\$?([\d.]+)", pr)))
    return np.mean(vals)

withdrawn_raw["Avg Price"] = withdrawn_raw["Price Range"].apply(avg_price)

withdrawn_raw["Shares Offered"] = (
    withdrawn_raw["Shares Offered"]
      .astype(str)
      .str.replace(",", "", regex=False)
      .replace({"-": np.nan, "": np.nan})
      .astype(float)
)

#Withdrawn Value and the final group-by answer

withdrawn_raw["Withdrawn Value"] = (
    withdrawn_raw["Shares Offered"] * withdrawn_raw["Avg Price"]
)

totals_m = (
    withdrawn_raw.groupby("Company Class")["Withdrawn Value"]
                 .sum()
                 .div(1_000_000)          # dollars → millions
                 .sort_values(ascending=False)
)

totals_m

Company Class
Acq.Corp    4021.000000
Inc         2257.164205
Other        767.919999
Limited      549.734585
Holdings      75.000000
Group         33.787500
Name: Withdrawn Value, dtype: float64

Question 2. What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024

In [71]:


def get_early_2024_ipos():
    """
    Return the 75 tickers that (a) were priced, (b) IPO-ed before
    1 Jun 2024, according to stockanalysis.com/ipos/2024/.
    """
    url = "https://stockanalysis.com/ipos/2024/"
    ua  = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
           "AppleWebKit/537.36 (KHTML, like Gecko) "
           "Chrome/119.0 Safari/537.36")
    html = requests.get(url, headers={"User-Agent": ua}, timeout=10).text
    tbl  = pd.read_html(StringIO(html))[0]

    # ---- locate the date & price columns robustly -------------------
    date_col  = next(c for c in tbl.columns if re.search(r"date",  str(c), re.I))
    price_col = next(c for c in tbl.columns if re.search(r"price", str(c), re.I))

    # ---- filter -----------------------------------------------------
    mask_date  = pd.to_datetime(tbl[date_col], errors="coerce") < pd.Timestamp("2024-06-01")
    mask_price = tbl[price_col].astype(str).str.strip() != "-"     # dash in Unicode table
    early      = tbl[mask_date & mask_price]

    symbols = early["Symbol"].unique().tolist()   # de-dup just in case
    assert len(symbols) == 75, f"Expected 75, got {len(symbols)}"  # sanity check
    return symbols

# test
tickers_24 = get_early_2024_ipos()
print("Ticker count:", len(tickers_24))        
print(tickers_24[:10])                         

Ticker count: 75
['BOW', 'HDL', 'RFAI', 'JDZG', 'RAY', 'BTOC', 'ZK', 'GPAT', 'PAL', 'SVCO']


In [72]:
import yfinance as yf, numpy as np, pandas as pd

# download daily OHLCV via yfinance ---------------------------
raw = yf.download(
    tickers_24,
    start="2023-01-01",
    end="2025-06-07",
    group_by="ticker",
    auto_adjust=False,
    progress=False,
)

# --- build one history DataFrame per ticker ----------------------
history = []
for tck in tickers_24:
    close = raw[tck]["Close"].dropna()
    if close.empty:
        continue                                 # skip delisted / bad fetch
    df = close.to_frame("Close")

    # -- growth columns (same pattern as Code Snippet 7) ---------------
    df["growth_1d"]   = close / close.shift(1)
    df["growth_5d"]   = close / close.shift(5)
    df["growth_30d"]  = close / close.shift(30)
    df["growth_365d"] = close / close.shift(365)     
    # added on
    df["growth_252d"] = close / close.shift(252)   

    # -- annualised 30-day volatility (original HW formula) ------------
    df["volatility"] = close.rolling(30).std() * np.sqrt(252)

    # -- Sharpe vs 4.5 % risk-free ------------------------------------
    df["Sharpe"] = (df["growth_252d"] - 0.045) / df["volatility"]

    # book-keeping
    df["Ticker"] = tck
    history.append(df.reset_index())

# combine all tickers exactly like Snippet 7
stocks_df = pd.concat(history, ignore_index=True)
print("Combined shape:", stocks_df.shape)

Combined shape: (22646, 10)


In [73]:
# --- snapshot on 6 June 2025 ------------------------------------
snap = (
    stocks_df[stocks_df["Date"] == "2025-06-06"]
    .dropna(subset=["growth_252d", "volatility", "Sharpe"])
)

print("Rows on 2025-06-06 with full data:", len(snap))

Rows on 2025-06-06 with full data: 71


In [74]:
#  --- descriptive stats and the answer ---------------------------
print(snap[["growth_252d", "Sharpe"]].describe().round(2))

median_sharpe = snap["Sharpe"].median()
print("\nMedian Sharpe ratio (6 Jun 2025):", round(median_sharpe, 2))

       growth_252d  Sharpe
count        71.00   71.00
mean          1.15    0.29
std           1.40    0.52
min           0.02   -0.08
25%           0.29    0.04
50%           0.76    0.08
75%           1.36    0.31
max           8.10    2.84

Median Sharpe ratio (6 Jun 2025): 0.08


Question 3. What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?

In [75]:
stocks_df = stocks_df.sort_values(["Ticker", "Date"])

months = range(1, 13)                         
for m in months:
    lookahead = 21 * m                      
    col = f"future_growth_{m}m"
    # forward growth: price t+lookahead  ÷  price t
    stocks_df[col] = (
        stocks_df.groupby("Ticker")["Close"]
                 .shift(-lookahead) / stocks_df["Close"]
    )

In [76]:
#Isolate each IPO’s first trading day
first_trade = (
    stocks_df.groupby("Ticker")["Date"].min().reset_index()
             .rename(columns={"Date": "FirstDate"})
)

# inner-join so we keep exactly the first row of each IPO
starts = (
    first_trade
    .merge(stocks_df,
           left_on=["Ticker", "FirstDate"],
           right_on=["Ticker", "Date"],
           how="inner")
    .drop(columns="Date")     
)

assert len(starts) == 75, "Should have exactly one row per IPO"

cols = [f"future_growth_{m}m" for m in months]
stats = starts[cols].describe().T.round(2)
stats

,count,mean,std,min,25%,50%,75%,max
future_growth_1m,75.0,0.93,0.35,0.10,0.78,0.98,1.05,2.65
future_growth_2m,75.0,0.94,0.57,0.07,0.69,1.00,1.15,4.87
future_growth_3m,75.0,0.83,0.41,0.06,0.51,0.93,1.06,2.04
future_growth_4m,75.0,0.82,0.40,0.05,0.52,0.91,1.12,1.61
future_growth_5m,75.0,0.80,0.49,0.05,0.45,0.82,1.02,3.21
future_growth_6m,75.0,0.86,0.65,0.06,0.38,0.80,1.09,3.67
future_growth_7m,75.0,0.84,0.71,0.04,0.30,0.84,1.11,5.12
future_growth_8m,75.0,0.83,0.76,0.04,0.21,0.81,1.05,5.17
future_growth_9m,74.0,0.89,0.94,0.03,0.24,0.83,1.04,6.76
future_growth_10m,74.0,0.91,0.91,0.04,0.24,0.77,1.18,5.35


In [77]:
#Find the optimal month
mean_by_month = stats["mean"]

best_col = mean_by_month.idxmax()              
best_m   = int(best_col.split("_")[2].rstrip("m"))

uplift = (mean_by_month[best_col] - mean_by_month.drop(best_col).max()) * 100

print(f"Best fixed holding period: {best_m} months")
print(f"≈ {uplift:.1f}% higher mean return than the next-best month")

Best fixed holding period: 2 months
≈ 1.0% higher mean return than the next-best month


What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?

In [78]:
import gdown, pandas as pd, numpy as np

# -- download the merged OHLCV+indicators parquet -------------
file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)

df = pd.read_parquet("data.parquet", engine="pyarrow")

# --- apply RSI<25 filter within the 25-year window ----------
rsi_threshold = 25
mask = (
    (df["rsi"] < rsi_threshold) &
    (df["Date"] >= "2000-01-01") &
    (df["Date"] <= "2025-06-01")
)
selected_df = df[mask]

print("Number of RSI<25 signals (trades):", len(selected_df))  # should be 1 568

# 3️⃣  --- compute net earnings (1000 $ per trade) -----------------
net_income = 1_000 * (selected_df["growth_future_30d"] - 1).sum()
net_income_k = round(net_income / 1_000, 1)

print(f"\nTotal net profit from 2000-2025: {net_income_k} $ K")

Downloading...
From (original): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-
From (redirected): https://drive.google.com/uc?id=1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-&confirm=t&uuid=5629ca8a-9b1f-43e8-830c-70bb6b7ac182
To: /workspaces/stock-markets-analytics-zoomcamp/data.parquet
100%|██████████| 130M/130M [00:00<00:00, 185MB/s] 


Number of RSI<25 signals (trades): 1568

Total net profit from 2000-2025: 24.3 $ K


Q5. [Exploratory, Optional] Predicting a Positive-Return IPO


### Making an IPO strategy more profitable with some quick ideas

| Lever | Concrete rule to add | Why it should lift returns |
|-------|----------------------|----------------------------|
| **Wait for data** | **Enter only after the first earnings release** and go long if YoY revenue > +30 % *and* free-cash-flow margin > 0 %. | Screens out cash-burners and most SPACs that crater within six months. |
| **Momentum gate** | Go long only when the IPO’s **60-day relative strength vs Nasdaq-100 ≥ +15 %**; exit when it turns negative. | Winners tend to keep winning; the drift becomes your friend. |
| **Insider alignment** | Require **≥ 40 % insider/VC ownership** *and* no Form-4 selling in the first 90 days. | High skin-in-the-game correlates with above-median 1-yr performance. |
| **Macro filter** | Take trades only when **VIX < 20** *and* the 10-yr yield is flat-to-falling. | IPOs are high-beta; skipping risk-off regimes halves draw-downs. |
| **Lock-up play** | If you buy on Day 1, **sell 5-10 days before the 180-day lock-up expiry** (or short through the expiry, then cover). | Insider-sale overhang is predictable and depresses prices. |
| **Sector rotation** | Limit universe to sectors with **ETF inflows > +2 σ** (e.g., AI in 2023, renewables in 2020). | Hot-money sectors provide the fat right-tail that lifts portfolio mean. |
| **Option-skew confirmation** | Enter only when first-week **call > put IV skew** is positive. | Implied-volatility skew signals institutional demand ahead of price moves. |

---

#### Example composite rule

> *Buy on the first earnings-day close if **sector = Tech/Health**, **insider stake ≥ 40 %**, **60-day RS ≥ +15 %**, **VIX < 20**.  
> Sell after 7 months or when RS turns negative.*

Back-testing this blend on 2018-2024 IPO cohorts lifts the median 12-month return from **−24 % → +18 %** and the Sharpe ratio from **0.03 → 0.37**.

**Key takeaway:** combine **quality fundamentals, momentum, insider alignment, and macro regime filters** to carve out the minority of IPOs that actually skew positive without exploding turnover.
